In [7]:
import os
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.embeddings import InfinityEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

## 定数定義

In [8]:
CHAT_MODEL_PATH = 'Qwen/Qwen3-4B-Thinking-2507'
EMBEDDING_MODEL_PATH = 'sbintuitions/sarashina-embedding-v2-1b'

DB_DIR = '/app/chroma_db'

In [9]:
def format_docs(docs):
    """
    検索された文書を文字列に変換
    クエリテンプレートが含まれている場合は、元の文書内容のみを抽出
    """
    formatted_docs = []
    for doc in docs:
        content = doc.page_content
        
        # クエリテンプレートが含まれている場合は、元の文書内容を抽出
        if content.startswith("text: "):
            # クエリテンプレート部分を除去
            original_content = content.replace("text: ", "")
            formatted_docs.append(original_content)
        else:
            formatted_docs.append(content)
    
    return "\n\n".join(formatted_docs)


def create_rag_chain(vectorstore, llm):
    """
    LCEL（LangChain Expression Language）でRAGチェーンを作成

    チェーンの流れ:
    質問 → 関連文書検索 → プロンプト作成 → LLM → 文字列出力
    """
    # 1. プロンプトテンプレートを作成
    prompt = ChatPromptTemplate.from_template("""以下の文脈情報を使用して質問に答えてください。

文脈情報:
{context}

質問: {question}

回答:""")

    # 2. 検索器（Retriever）を作成
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # 3. LCELでRAGチェーンを構築
    # | はパイプ演算子で、左の出力を右の入力に渡す
    rag_chain = (
        {
            "context": retriever | format_docs, # 質問→検索→文書をフォーマット
            "question": RunnablePassthrough() # 質問をそのまま通す
        }
        | prompt # 辞書→プロンプトに埋め込み
        | llm # プロンプト→LLMで処理
    )

    return rag_chain, retriever


def simple_rag_query(question, rag_chain, retriever):
    """
    LCELチェーンを使ったRAG処理
    """
    print(f"質問に関連する文書を検索中...")

    # 1. 関連文書を個別に取得（表示用）
    docs = retriever.invoke(question)
    print(f"{len(docs)}件の関連文書を見つけました")

    # 2. RAGチェーンを実行
    print("回答を生成中...")
    answer = rag_chain.invoke(question)
    print("回答生成完了!!")
    return answer.content, docs

In [10]:
%%time
embeddings = InfinityEmbeddings(
    model=EMBEDDING_MODEL_PATH,
    infinity_api_url='http://proxy',
)
vectorstore = Chroma(
    persist_directory=DB_DIR,
    embedding_function=embeddings
)

llm = ChatOpenAI(
    openai_api_key='EMPTY',
    openai_api_base='http://proxy/v1',
    model_name=CHAT_MODEL_PATH,
    temperature=0.1
)

rag_chain, retriever = create_rag_chain(vectorstore, llm)
# test_question = "task: クエリに関連した文章を検索してください \\n query: 生成AIとは何ですか？"
test_question = """
task: クエリに関連した文章を検索してください
query: AI開発におけるコスト削減方法は？
下記の例に従って回答してください。
```
## 朝に散歩をする
- 理由: 脳を活性化させるため
- 説明: 散歩をするとタスクを遂行する能力が向上する研究結果がある

## 毎日早く寝る
- 理由: 早く寝ると集中力がアップする
- 説明: 夜更かしする人とそうでない人とでは、前者の方が集中力がアップするというデータがある
```
また、
- なるべく多様な方法を紹介してください。
- 意味的に似た内容は避けてください。
"""
answer, sources = simple_rag_query(test_question, rag_chain, retriever)

質問に関連する文書を検索中...
3件の関連文書を見つけました
回答を生成中...
回答生成完了!!
CPU times: user 103 ms, sys: 118 ms, total: 221 ms
Wall time: 40.5 s


In [11]:
for source in sources:
    print('---')
    print(source.page_content)

---
text: 21 
a) 複数のリクエストを一つに集約する 
テキスト生成AI の Web API への複数リクエストを一つに集約すればコストカ
ットにつながる可能性がある。ただしこの手法ではリクエスト 1 件あたりのト
ークン数が多くなるため、リクエスト 1 件あたりの平均費用が高くなり、結果
としてコストカット効果に乏しい場合もある。また、テキスト生成AIにとって
の処理難易度が上がることで品質劣化を招く危険性もある。 
b) テキスト生成AI以外の手法を採用する 
例えば、文中の単語や特定の品詞を抽出するだけであれば形態素解析器で十
分であり、文間の意味的類似度を測るだけであればベクトル表現（ 埋め込み表
現、embedding ともいう）の採用で十分な場合もある。このようにテキスト生
成 AI 以外の手法で十分な場合ではテキスト生成 AI 以外の手法をまず検討すべ
きである。またこれらの自然言語処理手法は、テキスト生成AIとの二者択一で
はなく、相互に弱みを補完しながら組み合わせて利用することも十分に考えら
れる。 
c) 過去のテキスト生成AIの生成物を再利用する 
過去にテキスト生成AIで生成された実績のあるリクエストと同じようなリク
エスト（例えばリクエスト文字列をベクトル表現化し、それと近しいベクトル
表現の場合に、同じようなものとみなす）の場合、キャッシュサーバー等から
過去の生成物を送信するようにし、テキスト生成 AI の Web APIにリクエストを
させない。この手法をより極端にした場合、テキスト生成 AI の Web APIへのア
クセスは全て事前にバッチ処理で行い、テキスト生成AI関連の処理にかかるコ
ストを制御しやすくする手法も考えられる。この手法に関しては、 ５．２ ４） 
テスト済みの生成物のみを用いる場合の工夫 で詳細に述べる 
ウ テキスト生成 AI の機械学習モデルが直接提供されるケース 
大規模言語モデルの実行にともなう計算コストに直接影響される形でハード
ウェアや運用に関連するコストが増加する。そのため大規模言語モデルの実行
にともなう計算コストが重要であり、その観点や手法は Web API 利用のケース
での検討事項と同様である。 
3) 運用業務のコスト削減観点
---
text: 20 
当初想定した大規模言

In [12]:
print(answer)



## 過去の生成物をキャッシュで再利用する
- 理由: リクエストの重複を防ぎ、Web APIへのアクセスを削減するため
- 説明: 既に生成された実績のあるリクエストと類似のリクエストが来た場合、キャッシュサーバーから過去の生成物を再利用することで、テキスト生成AIのWeb APIにリクエストを送らない。これにより、リクエスト量が減少し、コストが削減される。

## テキスト生成AI以外のNLP手法を採用する
- 理由: テキスト生成AIのコストを回避し、効率的な処理を実現するため
- 説明: 文中の単語や品詞の抽出には形態素解析器、意味的類似度の測定にはベクトル表現（埋め込み表現）が十分な場合がある。これらの手法を用いることで、テキスト生成AIの利用を避け、コストを削減できる。

## RAG技術を導入する
- 理由: 関連文章の抽出により、大規模言語モデルの生成リクエストを削減するため
- 説明: 検索拡張生成（RAG）の技法では、質問文に対して関連文章を抽出し、それらをプロンプトとして大規模言語モデルに渡す。これにより、生成に必要なリクエストの数が減少し、コストが削減される。

## 機械的品質評価手法を導入する
- 理由: 人間による定性テストのコストと時間の増加を抑制するため
- 説明: 生成物の品質を機械的な手法（例: ベクトル距離の計算や自動チェック）で評価することで、人間のチェックを必要とせず、人件費や開発リードタイムを削減できる。
